In [1]:
import pyspark
from sklearn import preprocessing
import xgboost as xgb
import pandas as pd
import numpy as np
sc = pyspark.SparkContext('local[*]')

In [2]:
postulantes = pd.read_csv('Sources/postulantes_proc.csv').drop('Unnamed: 0', axis=1)
avisos = pd.read_csv('Sources/fiuba_6_avisos_detalle_limpio.csv').drop('Unnamed: 0', axis=1)
postulaciones = pd.read_csv('Sources/postulaciones_for_training.csv').drop('Unnamed: 0', axis=1)

In [3]:
postulaciones = postulaciones.merge(avisos, on='idaviso')
postulaciones = postulaciones.merge(postulantes, on='idpostulante')

In [4]:
postulaciones.drop(['titulo','descripcion'],axis=1, inplace=True)

In [5]:
test = pd.read_csv("Sources/test_final_100k.csv")

In [6]:
test = test.merge(avisos, on='idaviso')
test = test.merge(postulantes, on='idpostulante')
test['sepostulo']=1
test.drop(['titulo','descripcion'],axis=1, inplace=True)
test.head()

,id,idaviso,idpostulante,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario,fechanacimiento,sexo,sepostulo
0,0,739260,6M9ZQR,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,17,3.0,0.0,0.0,0.0,0.0,0.0,3.0,1976-03-29,FEM,1
1,1,739260,6v1xdL,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,17,0.0,0.0,3.0,0.0,0.0,3.0,0.0,1987-06-27,MASC,1
2,2,739260,ezRKm9,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,17,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1982-02-17,FEM,1
3,3,758580,1Q35ej,Full-time,Otro,Tecnologia / Sistemas,TeleTech,11,0.0,0.0,0.0,3.0,0.0,0.0,3.0,1949-12-24,MASC,1
4,4,758580,EAN4J6,Full-time,Otro,Tecnologia / Sistemas,TeleTech,11,0.0,0.0,0.0,0.0,0.0,2.0,2.0,1986-05-27,FEM,1


In [7]:
def dates_format(date):
    try:
        date_new = pd.to_datetime('today').year-pd.to_datetime(date).year
        date_new = int(date_new)
        return date_new
    except ValueError:
        return date

In [8]:
fechasRDD = sc.parallelize(postulaciones['fechanacimiento'])
postulaciones['fechanacimiento'] = fechasRDD.map(lambda x: dates_format(x)).collect()
fechasRDD = sc.parallelize(test['fechanacimiento'])
test['fechanacimiento'] = fechasRDD.map(lambda x: dates_format(x)).collect()
fechasRDD = None

In [9]:
resultado = postulaciones['sepostulo']
resultadob = test['sepostulo']
postulaciones.drop(['sepostulo','idaviso','idpostulante'],axis=1,inplace=True)
test.drop(['sepostulo','idaviso','idpostulante'],axis=1,inplace=True)
test.drop('id',axis=1,inplace=True)

In [10]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for col in postulaciones.columns:
    if not postulaciones[col].dtype == 'O': continue
    postulaciones[col] = postulaciones.apply(lambda x: str(x[col]),axis=1)
    test[col] = test.apply(lambda x: str(x[col]),axis=1)
    le.fit(pd.concat([postulaciones[col],test[col]]))
    test[col] = le.transform(test[col])
    postulaciones[col] = le.transform(postulaciones[col])
postulaciones.head()

,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario,fechanacimiento,sexo
0,1,4,120,2428,8,0.0,0.0,0.0,0.0,0.0,0.0,3.0,55,1
1,1,4,120,2428,8,0.0,0.0,0.0,0.0,0.0,3.0,2.0,45,2
2,1,2,36,3236,2,0.0,0.0,0.0,0.0,0.0,3.0,2.0,45,2
3,1,4,36,2228,8,0.0,0.0,0.0,0.0,0.0,3.0,2.0,45,2
4,1,4,64,154,14,0.0,0.0,0.0,0.0,0.0,3.0,2.0,45,2


In [11]:
postulaciones.sample(50)

,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario,fechanacimiento,sexo
9771631,1,4,14,2019,3,0.0,0.0,0.0,0.0,3.0,2.0,0.0,39,2
7212383,1,4,2,3183,2,0.0,0.0,0.0,3.0,0.0,3.0,0.0,50,2
11505473,1,4,185,647,5,0.0,0.0,0.0,0.0,3.0,0.0,0.0,37,2
7141728,1,4,141,203,6,0.0,0.0,0.0,0.0,0.0,0.0,3.0,44,2
5192830,1,2,42,1512,16,0.0,0.0,0.0,0.0,3.0,0.0,0.0,52,2
2536090,1,4,117,3183,10,0.0,0.0,0.0,2.0,0.0,0.0,0.0,45,1
12759098,2,3,14,1906,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50,1
11087484,1,4,74,234,15,0.0,0.0,0.0,0.0,3.0,0.0,0.0,37,1
1014738,1,4,166,4174,9,0.0,0.0,0.0,0.0,0.0,0.0,3.0,61,1
4898888,5,3,120,3577,12,3.0,0.0,0.0,0.0,0.0,3.0,3.0,59,1


In [12]:
postulaciones.rename(columns={x:int(y) for x,y in zip(postulaciones.columns,range(1,len(postulaciones.columns)+1))},inplace=True)
test.rename(columns={x:int(y) for x,y in zip(test.columns,range(1,len(test.columns)+1))},inplace=True)

In [19]:
aviso = None
postulantes = None
postulaciones.fillna(0,inplace=True)

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
X_train, X_test, y_train, y_test = train_test_split(postulaciones, resultado, test_size=0.2)
clf=RandomForestClassifier(n_estimators=100,n_jobs=-1)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)


In [22]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8515923962536817


In [26]:
feature_imp = pd.Series(clf.feature_importances_,index=postulaciones.columns).sort_values(ascending=False)
feature_imp

4     0.278616
3     0.213632
5     0.203473
13    0.138666
2     0.036429
12    0.030517
14    0.022892
1     0.019068
11    0.018379
10    0.017414
8     0.009466
9     0.006505
7     0.004301
6     0.000643
dtype: float64